no se especifico el tipo de archivos formato o caracteristicas del mismo como duracion, tasa de muestreo o canales. tampoco fue el mismo microfono ni se especifico alguno. Tampoco la distancia a la que s encontrara el microfono ni la intencidad de los hablantes. la edad de los participantes fue libre al igual que su sexo.

Hola a todos,
La carpeta compartida de Drive es la siguiente: https://drive.google.com/drive/folders/1mRfIfhgRez14NgRmaUUVMC5-yvXbKX5U?usp=drive_link
Encontrarán 4 carpetas: Audio, Imagen, Inercial, y fisiológico. Dentro de cada una de ellas deberán crear una subcarpeta con su nombre y podrán agregar ahí sus datos recopilados para cada una de las prácticas.
El formato para los nombres de los archivos en la primera práctica es el siguiente: PERSONA-METODO_NUM.
PERSONA es el nombre de ustedes para identificar los archivos individuales;
METODO modalidad de captura, ver datos de la práctica.
NUM es un número consecutivo para los audios recolectados.

In [122]:
import os
import warnings

import re
import shutil
import pandas as pd
import numpy as np

from pydub import AudioSegment
from pydub.utils import mediainfo
import librosa
import noisereduce as nr
from pydub.silence import detect_nonsilent
import random
from scipy.ndimage import shift
from scipy import stats
from scipy.signal import find_peaks
from tqdm.notebook import tqdm


# Librerías de Machine Learning de scikit-learn
from sklearn.model_selection import StratifiedShuffleSplit, cross_validate, train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.feature_selection import mutual_info_classif, SelectKBest, RFE
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
import joblib

warnings.filterwarnings("ignore", message="n_fft=.* is too large for input signal of length=.*", category=UserWarning, module="librosa")

In [92]:
PATH_AUDIO_RAW = r"C:\Users\Jesus\Downloads\Practice 1\DataSet\AUDIO\Raw"
PATH_AUDIO_WAV = r"C:\Users\Jesus\Downloads\Practice 1\DataSet\AUDIO\Wav"
PATH_AUDIO_PROCESSED = r"C:\Users\Jesus\Downloads\Practice 1\DataSet\AUDIO\Procesado"
PATH_AUDIO_PROCESSED_01 = r"C:\Users\Jesus\Downloads\Practice 1\DataSet\AUDIO\Procesado\metodo_01"
PATH_AUDIO_PROCESSED_02 = r"C:\Users\Jesus\Downloads\Practice 1\DataSet\AUDIO\Procesado\metodo_02"
PATH_AUDIO_PROCESSED_03 = r"C:\Users\Jesus\Downloads\Practice 1\DataSet\AUDIO\Procesado\metodo_03"


In [47]:
def convert_to_wav(input_file, output_file):
    """
    Convierte un archivo de audio a WAV estandarizado (44100 Hz, 16 bits, mono).

    Parámetros
    ----------
    input_file : str o AudioSegment
        Ruta del archivo de entrada o objeto AudioSegment.
    output_file : str
        Ruta del archivo WAV de salida.

    Retorna
    -------
    None
        Genera un archivo WAV en la ruta especificada.

    Ejemplos
    --------
    >>> convert_to_wav("entrada.mp3", "salida.wav")
    >>> convert_to_wav("entrada.m4a", "salida_mono.wav")
    """

    # Cargar el archivo de audio
    if isinstance(input_file, str):
        audio = AudioSegment.from_file(input_file)
    elif isinstance(input_file, AudioSegment):
        audio = input_file
    else:
        raise ValueError("input_file debe ser str o AudioSegment")

    # Estandarización de parámetros
    audio = (audio
             .set_frame_rate(44100)    # 44100 Hz
             .set_sample_width(2)      # 16 bits (2 bytes)
             .set_channels(1)          # 1 canal (mono)
            )

    # Parámetros de exportación
    export_params = {
        'format': 'wav',
        'codec': 'pcm_s16le',          # Códec PCM 16 bits little-endian
        'parameters': [
            '-ar', '44100',            # Tasa de muestreo
            '-ac', '1'                 # Canales
        ]
    }

    # Exportar archivo estandarizado
    audio.export(output_file, **export_params)

In [48]:
def convert_directory_to_wav(root_directory):
    audio_files = []
    
    for current_folder, subfolders, files in os.walk(root_directory):
        for file in files:
            full_path = os.path.join(current_folder, file)
            name, extension = os.path.splitext(file)
            if extension.lower() in ['.mp3', '.aac', '.flac', '.ogg', '.m4a']:
                audio_files.append((full_path, name))
    
    progress_bar = tqdm(audio_files, desc="Convirtiendo archivos", unit="archivo")
    for full_path, name in progress_bar:
        try:
            progress_bar.set_postfix(archivo=name)
            output_file = os.path.join(PATH_AUDIO_WAV, f"{name}.wav")
            convert_to_wav(full_path, output_file)
        except Exception as e:
            print(f"Error al procesar {full_path}: {e}")

In [38]:
def analyze_audio_dataset(root_dir):
    """
    Analiza archivos de audio en un directorio y subdirectorios, extrayendo metadatos básicos.

    Recorre recursivamente todas las carpetas desde el directorio raíz y para cada archivo de audio
    compatible, extrae información como tamaño, duración, formato, tasa de muestreo y canales.

    Parámetros
    ----------
    root_dir : str
        Directorio raíz que contiene los archivos de audio a analizar.
        Debe ser una ruta válida accesible por el sistema.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con los siguientes campos por cada archivo:
        - file_path: Ruta completa del archivo
        - file_size: Tamaño en bytes
        - format: Extensión/Formato del archivo (WAV, MP3, M4A, etc.)
        - duration: Duración en segundos
        - sample_rate: Tasa de muestreo en Hz
        - channels: Número de canales de audio

    Notas
    -----
    - Depende de FFmpeg o libav para manejar formatos como M4A, MP3, etc.
    - Archivos corruptos o no soportados serán omitidos en los resultados.
    - El tamaño del archivo se obtiene del sistema de archivos, no de los metadatos de audio.

    Ejemplos
    --------
    >>> df = analyze_audio_dataset('audio_dataset/')
    >>> df[['format', 'duration', 'sample_rate']].head()
    >>> print(df.describe())
    """


    audio_data = []
    audio_files = []
    
    for current_dir, _, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(current_dir, file)
            audio_files.append(file_path)
    
    progress_bar = tqdm(audio_files, desc="Analizando archivos de audio", unit="archivo")
    
    for file_path in progress_bar:
        try:
            file_size = os.path.getsize(file_path)
            file_format = os.path.splitext(file_path)[1][1:].upper()
            
            audio = AudioSegment.from_file(file_path)
            audio_info = mediainfo(file_path)
            
            audio_metadata = {
                'file_path': file_path,
                'file_size': file_size,
                'format': file_format,
                'duration': round(len(audio) / 1000.0, 3),
                'sample_rate': int(audio_info.get('sample_rate', 0)),
                'channels': int(audio_info.get('channels', 1)),
            }
            
            audio_data.append(audio_metadata)
            progress_bar.set_postfix(archivo=os.path.basename(file_path))
        
        except Exception as e:
            print(f"Error procesando {file_path}: {str(e)}")
            continue
    
    return pd.DataFrame(audio_data)

In [39]:
df = analyze_audio_dataset(PATH_AUDIO_RAW)

Analizando archivos de audio:   0%|          | 0/545 [00:00<?, ?archivo/s]

Error procesando C:\Users\Jesus\Downloads\Practice 1\DataSet\AUDIO\Raw\David-Murillo\Metodo-01.rar: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 2.2.2 Copyright (c) 2000-2014 the FFmpeg developers
  built on May 22 2014 19:56:44 with gcc 4.8.2 (GCC)
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --disable-w32threads --enable-avisynth --enable-bzlib --enable-fontconfig --enable-frei0r --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libcaca --enable-libfreetype --enable-libgme --enable-libgsm --enable-libilbc --enable-libmodplug --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-librtmp --enable-libschroedinger --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvo-aacenc --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-lib

In [40]:
print(df.describe())

           file_size    duration   sample_rate    channels
count     544.000000  544.000000    544.000000  544.000000
mean    39617.128676    2.420697  42512.867647    1.193015
std     44151.710450    0.781631  10745.885326    0.395028
min      2691.000000    0.814000  16000.000000    1.000000
25%      9171.000000    1.904000  44100.000000    1.000000
50%     22767.500000    2.299000  48000.000000    1.000000
75%     61347.250000    2.816000  48000.000000    1.000000
max    247544.000000    8.192000  48000.000000    2.000000


In [42]:
column_list = df.columns.values.tolist()
for column_name in column_list:
    print(column_name,":",df[column_name].unique())

file_path : ['C:\\Users\\Jesus\\Downloads\\Practice 1\\DataSet\\AUDIO\\Raw\\Cielo-Higuera\\Practica-01_Audio\\Cielo-01_01.m4a'
 'C:\\Users\\Jesus\\Downloads\\Practice 1\\DataSet\\AUDIO\\Raw\\Cielo-Higuera\\Practica-01_Audio\\Cielo-01_02.m4a'
 'C:\\Users\\Jesus\\Downloads\\Practice 1\\DataSet\\AUDIO\\Raw\\Cielo-Higuera\\Practica-01_Audio\\Cielo-01_03.m4a'
 'C:\\Users\\Jesus\\Downloads\\Practice 1\\DataSet\\AUDIO\\Raw\\Cielo-Higuera\\Practica-01_Audio\\Cielo-01_04.m4a'
 'C:\\Users\\Jesus\\Downloads\\Practice 1\\DataSet\\AUDIO\\Raw\\Cielo-Higuera\\Practica-01_Audio\\Cielo-01_05.m4a'
 'C:\\Users\\Jesus\\Downloads\\Practice 1\\DataSet\\AUDIO\\Raw\\Cielo-Higuera\\Practica-01_Audio\\Cielo-02_01.m4a'
 'C:\\Users\\Jesus\\Downloads\\Practice 1\\DataSet\\AUDIO\\Raw\\Cielo-Higuera\\Practica-01_Audio\\Cielo-02_02.m4a'
 'C:\\Users\\Jesus\\Downloads\\Practice 1\\DataSet\\AUDIO\\Raw\\Cielo-Higuera\\Practica-01_Audio\\Cielo-02_03.m4a'
 'C:\\Users\\Jesus\\Downloads\\Practice 1\\DataSet\\AUDIO\\Raw\\Ciel

In [49]:
convert_directory_to_wav(PATH_AUDIO_RAW)

Convirtiendo archivos:   0%|          | 0/495 [00:00<?, ?archivo/s]

In [78]:
def reduce_noise(audio):
    samples = np.array(audio.get_array_of_samples())
    reduced_noise = nr.reduce_noise(y=samples, sr=audio.frame_rate)
    reduced_audio = AudioSegment(
        reduced_noise.tobytes(), 
        frame_rate=audio.frame_rate,
        sample_width=audio.sample_width,
        channels=audio.channels
    )
    return reduced_audio

def trim_silence(audio, file_path, silence_thresh=-55, min_silence_len=1200):
    non_silent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    if non_silent_ranges:
        start_trim = non_silent_ranges[0][0]
        end_trim = non_silent_ranges[-1][1]
        trimmed_audio = audio[start_trim:end_trim]
    else:
        trimmed_audio = audio
        #print(f"No se detectó silencio significativo en {file_path}:")
        #audio = ipd.Audio(file_path)
        #display(audio)
    return trimmed_audio

def normalize_audio(audio, target_dBFS=-20.0):
    change_in_dBFS = target_dBFS - audio.dBFS
    normalized_audio = audio.apply_gain(change_in_dBFS)
    return normalized_audio

def dynamic_compression(audio, threshold=-20.0, ratio=4.0, attack=5.0, release=50.0):
    compressed_audio = audio.compress_dynamic_range(
        threshold=threshold,
        ratio=ratio,
        attack=attack,
        release=release
    )
    return compressed_audio


def process_audio_files(root_directory):
    # Obtener la lista de todos los archivos de audio
    audio_files = []
    for current_folder, subfolders, files in os.walk(root_directory):
        for file in files:
            full_path = os.path.join(current_folder, file)
            audio_files.append(full_path)



    progress_bar = tqdm(audio_files, desc="Procesando archivos de audio", unit="archivo")
    for full_path in progress_bar:
        name, extension = os.path.splitext(os.path.basename(full_path))
        try:
            # Aplicar cada paso del procesamiento
            audio = AudioSegment.from_file(full_path)
            audio = reduce_noise(audio)
            audio = trim_silence(audio, full_path)
            #audio = normalize_audio(audio)
            #audio = dynamic_compression(audio)

            # Guardar el archivo final procesado y convertido
            output_file = os.path.join(PATH_AUDIO_PROCESSED, f"{name}{extension}")
            audio.export(output_file)
            # Actualizar la barra de progreso con el nombre del archivo actual
            progress_bar.set_postfix(archivo=name)  # Muestra el nombre del archivo actual
        except Exception as e:
            print(f"Error al procesar {full_path}: {e}")

In [79]:
process_audio_files(PATH_AUDIO_WAV)

Procesando archivos de audio:   0%|          | 0/480 [00:00<?, ?archivo/s]

In [80]:
df = analyze_audio_dataset(PATH_AUDIO_PROCESSED)

Analizando archivos de audio:   0%|          | 0/480 [00:00<?, ?archivo/s]

In [81]:
print(df.describe())

         file_size    duration  sample_rate  channels
count    480.00000  480.000000        480.0     480.0
mean   17729.87500    2.162840      44100.0       1.0
std     4626.19486    0.578263          0.0       0.0
min     5451.00000    0.628000      44100.0       1.0
25%    14802.75000    1.797250      44100.0       1.0
50%    17572.00000    2.143000      44100.0       1.0
75%    20707.00000    2.535000      44100.0       1.0
max    35126.00000    4.337000      44100.0       1.0


In [85]:
def separate_files_by_method(root_directory):
    """
    Separa archivos de audio en subcarpetas según el método (01, 02, 03).

    Se examinan todos los archivos en el directorio raíz y, si el archivo es un archivo WAV 
    y contiene un patrón que indica el método (por ejemplo, xxxx-01_xx), se mueve a una subcarpeta 
    correspondiente a ese método. Si el patrón no se encuentra, se informa al usuario.

    Parámetros
    ----------
    root_directory : str
        Directorio donde se encuentran los archivos de audio.

    Retorna
    -------
    None

    Notas
    -----
    Se crean subcarpetas en el mismo directorio raíz para cada método encontrado, 
    nombradas como 'metodo_XX'.

    Examples
    --------
    >>> main_directory = "/ruta/a/tu/directorio/de/audios"
    >>> separate_files_by_method(main_directory)
    Archivo 'ejemplo.wav' movido a 'metodo_01'
    """
    for file in os.listdir(root_directory):
        if file.endswith(".wav"):  # Asegúrate que solo procese archivos WAV
            match = re.search(r"(0[1-3])[-_]", file)  # Busca el patrón xxxx-01_xx, xxxx-02-xx, etc.
            if match:
                method_number = match.group(1)  # Extrae el número del método (01, 02, 03)
                destination_folder = os.path.join(root_directory, f"metodo_{method_number}")

                # Crea la carpeta si no existe
                os.makedirs(destination_folder, exist_ok=True)

                source_file_path = os.path.join(root_directory, file)
                destination_file_path = os.path.join(destination_folder, file)

                # Mueve el archivo a la carpeta correspondiente
                shutil.move(source_file_path, destination_file_path)
            else:
                print(f"Archivo '{file}' no coincide con el patrón y no fue movido.")

In [86]:
separate_files_by_method(PATH_AUDIO_PROCESSED)

# Extracción de características
## Metodo 1

In [101]:
def calcular_energia_banda(y, sr, banda_inferior, banda_superior):
    frame_length = 128  # Un valor común, puedes ajustarlo
    hop_length = frame_length // 4  # Un valor común, puedes ajustarlo
    n_fft = frame_length  # Debe ser mayor o igual que frame_length

    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))

    # Obtiene las frecuencias correspondientes a cada fila de S
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

    # Encuentra los índices de las filas que se encuentran dentro de la banda
    indices = np.where((freqs >= banda_inferior) & (freqs <= banda_superior))[0]

    # Calcula la energía en la banda
    energia_en_banda = np.mean(librosa.feature.rms(S=S[indices]))

    return energia_en_banda

In [118]:
def extraer_caracteristicas(y_aug, sr_aug):
    # Carga del audio
    y, sr = y_aug, sr_aug
    
    # Dominio del tiempo
    feats = {}
    feats['rms'] = np.sqrt(np.mean(y**2))
    feats['tasa_cruces_cero'] = librosa.feature.zero_crossing_rate(y).mean()
    feats['proporcion_silencio'] = np.mean(librosa.feature.rms(y=y) < 0.01)
    feats['amplitud_maxima'] = np.max(np.abs(y))
    feats['amplitud_minima'] = np.min(np.abs(y))
    feats['amplitud_media'] = np.mean(np.abs(y))
    feats['varianza_amplitud'] = np.var(y)
    feats['asimetria_amplitud'] = stats.skew(y)
    feats['curtosis_amplitud'] = stats.kurtosis(y)
    feats['mediana_amplitud'] = np.median(np.abs(y))
    feats['rango_intercuartilico'] = stats.iqr(y)
    
    # Pico de la autocorrelación
    autocorr = np.correlate(y, y, mode='full')
    peaks, _ = find_peaks(autocorr)
    feats['pico_autocorrelacion'] = np.max(autocorr[peaks]) if peaks.size > 0 else 0

    # Cruces de umbral (umbral = 0)
    feats['cruces_umbral'] = len(np.where(np.diff(y > 0))[0])
    
    # Entropía temporal
    hist, _ = np.histogram(y, bins=20, density=True)
    feats['entropia_temporal'] = stats.entropy(hist)
    
    # Dominio frecuencial
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    feats['centroide_espectral_media'] = np.mean(centroid)
    feats['centroide_espectral_varianza'] = np.var(centroid)
    feats['centroide_espectral_asimetria'] = stats.skew(centroid[0])
    feats['centroide_espectral_curtosis'] = stats.kurtosis(centroid[0])
    
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    feats['ancho_banda_media'] = np.mean(bandwidth)
    feats['ancho_banda_varianza'] = np.var(bandwidth)
    
    feats['rolloff_espectral_media'] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    feats['planitud_espectral_media'] = np.mean(librosa.feature.spectral_flatness(y=y))
    feats['contrast_espectral_media'] = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))
    
    
    # MFCC (1-13)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    for i, mfcc in enumerate(mfccs, start=1):
        feats[f'mfcc{i}_media'] = np.mean(mfcc)
    
    # Características Croma (1-12)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    for i, c in enumerate(chroma, start=1):
        feats[f'chroma{i}_media'] = np.mean(c)
    
    # Tonnetz (1-6)
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
    for i, t in enumerate(tonnetz, start=1):
        feats[f'tonnetz{i}_media'] = np.mean(t)
    
    S = np.abs(librosa.stft(y))   
   
    feats['coeficiente_polinomico0'] = np.mean(librosa.feature.poly_features(S=S, order=0))

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    feats['densidad_onset'] = np.mean(onset_env)
    
    return feats

In [96]:
def data_augmentation(y, sr, tecnicas=['ruido', 'desplazamiento', 'cambio_velocidad', 'cambio_tono', 'estiramiento_tiempo']):
    datos_aumentados = [(y, sr)]  # Incluye los datos originales

    for tecnica in tecnicas:
        if tecnica == 'ruido':
            ruido = np.random.randn(len(y))
            y_aumentado = y + 0.05 * ruido  # Ajusta la intensidad del ruido
            datos_aumentados.append((y_aumentado, sr))
        elif tecnica == 'desplazamiento':
            desplazamiento = random.randint(-len(y) // 5, len(y) // 5)  # Desplazamiento aleatorio
            y_aumentado = shift(y, desplazamiento, cval=0.0)
            datos_aumentados.append((y_aumentado, sr))
        elif tecnica == 'cambio_velocidad':
            rate  = random.uniform(0.9, 1.1)  # Factor de cambio de velocidad aleatorio
            y_aumentado = librosa.effects.time_stretch(y, rate=rate)
            datos_aumentados.append((y_aumentado, sr))
        elif tecnica == 'cambio_tono':
            n_steps = random.randint(-2, 2)  # Número de semitonos a desplazar
            y_aumentado = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=n_steps)
            datos_aumentados.append((y_aumentado, sr))
        elif tecnica == 'estiramiento_tiempo':
            rate = random.uniform(0.8, 1.2)  # Factor de estiramiento del tiempo
            y_aumentado = librosa.effects.time_stretch(y, rate=rate)
            datos_aumentados.append((y_aumentado, sr))

    return datos_aumentados

In [104]:
def procesar_archivos(ruta):
    data = []
    # Buscar todos los archivos de audio en la ruta (incluye subdirectorios)
    archivos = librosa.util.find_files(ruta)
    
    for archivo in archivos:
        # Obtener nombre del archivo sin ruta ni extensión
        nombre_completo = os.path.splitext(os.path.basename(archivo))[0]
        
        # Extraer etiqueta (parte antes del primer '-')
        etiqueta = nombre_completo.split('-')[0]
        
        # Procesar audio
        y, sr = librosa.load(archivo)
        datos_aumentados = data_augmentation(y, sr)
        
        for y_aug, sr_aug in datos_aumentados:
            caracteristicas = extraer_caracteristicas(y_aug, sr_aug)
            
            # Agregar metadatos
            caracteristicas['etiqueta'] = etiqueta  # Etiqueta limpia
            
            data.append(caracteristicas)
    
    return pd.DataFrame(data)

In [124]:
pd.DataFrame(procesar_archivos(PATH_AUDIO_PROCESSED_01)).to_csv(f"dataset_01.csv", index=False)
pd.DataFrame(procesar_archivos(PATH_AUDIO_PROCESSED_02)).to_csv(f"dataset_02.csv", index=False)
pd.DataFrame(procesar_archivos(PATH_AUDIO_PROCESSED_03)).to_csv(f"dataset_03.csv", index=False)

# Seleccion de caracteristicas

In [ ]:
def select_feature(df):

    # Separar características (X) y etiquetas (y)
    X = df.drop(columns=['etiqueta'])
    y = df['etiqueta']
    feature_names = X.columns


    # Normalización Min-Max
    def normalize_scores(scores):
        return MinMaxScaler().fit_transform(np.array(scores).reshape(-1, 1)).flatten()


    # Función genérica para aplicar una técnica
    def apply_technique(name, func, *args, **kwargs):
        scores = func(*args)
        return {
            'name': name,
            'scores': normalize_scores(scores),    }

    # Técnicas individuales
    def mutual_information(X, y):
        selector = SelectKBest(score_func=mutual_info_classif, k='all')
        selector.fit(X, y)
        return selector.scores_

    def lightgbm_importance(X, y):
        model = lgb.LGBMClassifier(verbosity=-1)
        model.fit(X, y)
        return model.feature_importances_

    def rfe_ranking(X, y):
        model = RandomForestClassifier()
        selector = RFE(estimator=model, n_features_to_select=5)
        selector.fit(X, y)
        return -selector.ranking_  # Invertimos el ranking

    def permutation_importance_scores(X, y):
        model = RandomForestClassifier()
        model.fit(X, y)
        perm_importance = permutation_importance(model, X, y, scoring='accuracy', n_repeats=10, random_state=42)
        return perm_importance.importances_mean

    # Aplicar técnicas
    techniques = []
    with tqdm([
        ('Mutual Information', mutual_information),
        ('LightGBM Importance', lightgbm_importance),
        ('RFE Ranking', rfe_ranking),
        ('Permutation Importance', permutation_importance_scores)
    ], desc="Aplicando técnicas") as progress:
        for name, func in progress:
            progress.set_description(f"Aplicando técnica {name}")  # Actualiza el mensaje en tqdm
            techniques.append(apply_technique(name, func, X, y, feature_names=feature_names))

    # Combinar resultados 
    combined_scores = sum(t['scores'] for t in techniques) / len(techniques)
    df = pd.DataFrame({'Feature': feature_names, 'Combined Scores': combined_scores})

    # Ordenar de mayor a menor según 'Combined Scores'
    df_ordenado = df.sort_values('Combined Scores', ascending=False)

    return df_ordenado

In [128]:
# Carga el dataset completo desde un archivo CSV
df_01 = pd.read_csv("dataset_01.csv")
df_02 = pd.read_csv("dataset_02.csv")
df_03 = pd.read_csv("dataset_03.csv")

In [ ]:
# Selecciona las mejores features de cada dataset
df_feature_01 = select_feature(df_01)
df_feature_02 = select_feature(df_02)
df_feature_03 = select_feature(df_03)

In [ ]:
df_feature_01[]

In [ ]:
df_feature_02

In [ ]:
df_feature_03

In [131]:
def entrenar_modelos(df, target_col, nombre_modelo):
    # Separar características y etiquetas del DataFrame
    X = df.drop(columns=[target_col])
    labels = df[target_col]
   
    # Normalizar los datos
    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(X)

    # Codificar etiquetas
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    # Dividir los datos
    X_train, X_test, y_train, y_test = train_test_split(
        X_normalized, y, test_size=0.2, random_state=42
    )

    # Configuración común
    cv_folds = 5
    scoring = ['accuracy', 'f1_macro']

    # Entrenar y evaluar SVM
    print("\n=== SVM ===")
    svm_model = SVC()
    svm_cv = cross_validate(svm_model, X_train, y_train, cv=cv_folds, scoring=scoring)
    print_metricas(svm_cv)
    
    svm_model.fit(X_train, y_train)
    guardar_modelo(svm_model, f"{nombre_modelo}_svm")
    evaluar_modelo(svm_model, X_test, y_test, label_encoder)

    # Entrenar y evaluar KNN
    print("\n=== KNN ===")
    knn_model = KNeighborsClassifier()
    knn_cv = cross_validate(knn_model, X_train, y_train, cv=cv_folds, scoring=scoring)
    print_metricas(knn_cv)
    
    knn_model.fit(X_train, y_train)
    guardar_modelo(knn_model, f"{nombre_modelo}_knn")
    evaluar_modelo(knn_model, X_test, y_test, label_encoder)

    # Entrenar y evaluar CatBoost
    print("\n=== CatBoost ===")
    catboost_model = CatBoostClassifier(iterations=100, verbose=0)
    catboost_cv = cross_validate(catboost_model, X_train, y_train, cv=cv_folds, scoring=scoring)
    print_metricas(catboost_cv)
    
    catboost_model.fit(X_train, y_train)
    guardar_modelo(catboost_model, f"{nombre_modelo}_catboost")
    evaluar_modelo(catboost_model, X_test, y_test, label_encoder)
    

def print_metricas(resultados_cv):
    print(f"CV Accuracy: {np.mean(resultados_cv['test_accuracy']):.2f} ± {np.std(resultados_cv['test_accuracy']):.2f}")
    print(f"CV F1 Macro: {np.mean(resultados_cv['test_f1_macro']):.2f} ± {np.std(resultados_cv['test_f1_macro']):.2f}")

def guardar_modelo(modelo, nombre):
    joblib.dump(modelo, f'{nombre}.pkl')
    print(f"Modelo guardado como: {nombre}.pkl")

def evaluar_modelo(modelo, X_test, y_test, label_encoder):
    y_pred = modelo.predict(X_test)
    print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
    print("Test F1 Macro:", f1_score(y_test, y_pred, average='macro'))
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [133]:
entrenar_modelos(df_01, "etiqueta", "01")


=== SVM ===
CV Accuracy: 0.81 ± 0.01
CV F1 Macro: 0.84 ± 0.02
Modelo guardado como: 01_svm.pkl

Test Accuracy: 0.828125
Test F1 Macro: 0.8627237851407477
                 precision    recall  f1-score   support

         AdriaM       1.00      1.00      1.00         1
           AleM       1.00      0.71      0.83         7
  AndresCalzada       1.00      1.00      1.00        11
         Ariana       1.00      1.00      1.00         3
          BetoM       1.00      0.80      0.89         5
          Bruce       1.00      1.00      1.00         4
         Camila       1.00      0.75      0.86         4
          Cielo       1.00      0.50      0.67         6
         DafneA       1.00      0.86      0.92         7
         Daniel       1.00      0.88      0.93         8
          David       1.00      1.00      1.00         6
         Didier       1.00      0.83      0.91         6
            Eri       1.00      0.80      0.89         5
           Erik       1.00      1.00      1.00

In [134]:
entrenar_modelos(df_02, "etiqueta", "02")
entrenar_modelos(df_03, "etiqueta", "03")


=== SVM ===
CV Accuracy: 0.81 ± 0.01
CV F1 Macro: 0.84 ± 0.01
Modelo guardado como: 02_svm.pkl

Test Accuracy: 0.8385416666666666
Test F1 Macro: 0.8469731873532647
                 precision    recall  f1-score   support

         AdriaM       0.50      1.00      0.67         1
           AleM       1.00      0.71      0.83         7
  AndresCalzada       1.00      1.00      1.00        11
         Ariana       0.50      1.00      0.67         3
          BetoM       1.00      0.80      0.89         5
          Bruce       0.24      1.00      0.38         4
         Camila       1.00      0.75      0.86         4
          Cielo       1.00      0.83      0.91         6
         DafneA       1.00      0.71      0.83         7
         Daniel       1.00      0.88      0.93         8
          David       1.00      1.00      1.00         6
         Didier       1.00      0.83      0.91         6
            Eri       1.00      0.80      0.89         5
           Erik       1.00      1.00

c:\Users\Jesus\.conda\envs\DataScience\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


CV Accuracy: 0.77 ± 0.02
CV F1 Macro: 0.79 ± 0.02
Modelo guardado como: 03_svm.pkl

Test Accuracy: 0.796875
Test F1 Macro: 0.7944498435548252


c:\Users\Jesus\.conda\envs\DataScience\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Jesus\.conda\envs\DataScience\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Jesus\.conda\envs\DataScience\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

                 precision    recall  f1-score   support

         AdriaM       0.04      1.00      0.08         1
           AleM       1.00      0.71      0.83         7
  AndresCalzada       1.00      1.00      1.00        11
         Ariana       0.60      1.00      0.75         3
          BetoM       0.80      0.80      0.80         5
          Bruce       0.67      1.00      0.80         4
         Camila       1.00      0.75      0.86         4
          Cielo       1.00      0.83      0.91         6
         DafneA       1.00      0.86      0.92         7
         Daniel       1.00      0.88      0.93         8
          David       1.00      1.00      1.00         6
         Didier       1.00      0.83      0.91         6
            Eri       1.00      0.80      0.89         5
           Erik       0.50      1.00      0.67         1
         Esther       1.00      0.83      0.91         6
       Fernando       1.00      0.67      0.80         6
      Francisco       1.00    

In [ ]:
def cargar_modelo(ruta_modelo):
    """
    Carga un modelo guardado desde un archivo .pkl

    Parámetros:
    -----------
    ruta_modelo : str
        Ruta completa al archivo .pkl del modelo guardado

    Retorna:
    --------
    model
        El modelo cargado
    """
    try:
        modelo = joblib.load(ruta_modelo)
        print(f"Modelo cargado correctamente desde: {ruta_modelo}")
        return modelo
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta {ruta_modelo}")
    except Exception as e:
        print(f"Error al cargar el modelo: {str(e)}")